## cross-LLM model comparisons


In [1]:
llm_models = [
    "qwen2.5", "llama3.1", "mistral", "nemotron-mini",
    "phi3.5", "gemma2"
]

In [2]:
from time import time
from datetime import datetime
import re
from pathlib import Path

import pandas as pd 

import ollama

from utils import *

In [3]:
pattern = r'```json\s*(.*?)\s*```'
file_log = "cross-llm-compare.log"
zi =  "person" # "字"  # "心"

lang = "ENU"
try:
    prompt_zi = LLM_PROMPT_ZI_DICT[lang]
except Exception as e:
    raise Exception(str(e))

In [4]:
for model_name in llm_models:
    fp = Path(f"./llm_models/{lang}") / f"{lang}-{zi}-{model_name}-1.txt"    
    if fp.exists():
        log_msg(f"SKIP {zi} - {model_name} : Already processed - {fp} ", file_log)
        continue

    print(f"Calling '{model_name}' on '{zi}' ...")
    
    ts_now = datetime.now()
    ts_1 = time()
    try:
        resp = ollama.generate(model=model_name, prompt=prompt_zi.format(zi=zi))
    except Exception as e:
        log_msg(f"[ERROR] {zi} - {model_name} : Failed to call ollama.generate()", file_log)
        continue
        
    ts_2 = time()
    del_t = ts_2 - ts_1
    del_t_str = f"{del_t:.3f}"
    json_raw = resp.get('response', "")
    if not json_raw: 
        log_msg(f"[ERROR] No LLM response: {zi} - {model_name}", file_log)
        continue

    log_msg(f"[{str(ts_now)}] Completed {lang} - {zi} - {model_name} in {del_t_str} sec")
    with open(fp, "w", encoding="utf-8") as f:
        f.write(json_raw)

    
    # match = re.search(pattern, json_raw, re.DOTALL)

    # if match:
    #     json_string = match.group(1)
    #     if not json_string:
    #         log_msg(f"[WARN] No JSON extracted: {zi}")
    #         continue
            
    #     with open(fp, "w", encoding="utf-8") as f:
    #         f.write(json_string)
    # else:
    #     log_msg(f"[ERROR] No JSON extracted: {zi}", file_log)

Calling qwen2.5 on person ...
[2024-09-21 12:15:31.842667] Completed ENU - person - qwen2.5 in 116.013 sec
Calling llama3.1 on person ...
[2024-09-21 12:17:27.856358] Completed ENU - person - llama3.1 in 156.096 sec
Calling mistral on person ...
[2024-09-21 12:20:03.953041] Completed ENU - person - mistral in 235.236 sec
Calling nemotron-mini on person ...
[2024-09-21 12:23:59.189920] Completed ENU - person - nemotron-mini in 77.266 sec
Calling phi3.5 on person ...
[2024-09-21 12:25:16.457514] Completed ENU - person - phi3.5 in 116.952 sec
Calling gemma2 on person ...
[2024-09-21 12:27:13.410975] Completed ENU - person - gemma2 in 242.190 sec
